In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Calculate unbalance score

In [0]:
import numpy as np
import pandas as pd

In [0]:
# district: name of the district(str)
# len_period: # of days scoring(int)(1~38)
# weights: weight infos for each score(dict)
def getUnbalanceScore(district, len_period, weights):
  data_path = './Data/district_0314-0512/'
  df = pd.read_csv(data_path+district+'_0314-0512.csv', header=0, parse_dates=['collect_at', 'stock_at'], engine='python', encoding='utf-8-sig')
  # missing data handling
  df.dropna(inplace=True)
  # select columns
  df = df[['code', 'stock_at', 'collect_at', 'remain_stat']]
  # only business days
  df = df[df.collect_at.dt.weekday.isin([0, 1, 2, 3, 4])]
  # counting point: 23:55
  df = df[df.collect_at.dt.time == pd.datetime.datetime.time(pd.to_datetime('23:55:00'))]  
  # set collect_at as an index
  df = df.set_index('collect_at')

  # Score each day
  # decide scoring period
  scoring_period = df.index[-len_period:]
  workday_index = sorted(set([idx.date() for idx in scoring_period]))
  score_df = pd.DataFrame(index=workday_index, columns=['shortage_score', 'overstock_score', 'unbalance_score'])

  new_idx = sorted(set(scoring_period))
  for idx in new_idx:
    daily_df = df[df.index == idx]
    num_valid = len(daily_df) - len(daily_df[daily_df['remain_stat']=='break'])

    shortage_score = len(daily_df[daily_df['remain_stat'].isin(['empty', 'few'])]) / num_valid
    overstock_score = len(daily_df[daily_df['remain_stat']=='plenty']) / num_valid
    #weighted average
    unbalance_score = weights['shortage']*shortage_score + weights['overstock']*overstock_score
    score_df.loc[idx] = [shortage_score, overstock_score, unbalance_score]

  return score_df.mean()

In [0]:
from tqdm import tqdm

districts = ['강남구','강동구','강북구','강서구','관악구','광진구','구로구','금천구',
             '노원구','도봉구','동대문구','동작구','마포구','서대문구','서초구','성동구',
             '성북구','송파구','양천구','영등포구','용산구','은평구','종로구','중구','중랑구']
weights = {'shortage': 0.8,
           'overstock': 0.2}
len_period = 30
scores = pd.DataFrame(index=districts, columns=['shortage_score', 'overstock_score', 'unbalance_score'])

for district in tqdm(districts):
  scores.loc[district] = getUnbalanceScore(district=district, len_period=len_period, weights=weights)

100%|██████████| 25/25 [10:02<00:00, 24.11s/it]


In [0]:
scores.index.name = 'District'

In [0]:
sorted_df = scores.sort_values(by=['unbalance_score', 'shortage_score'], ascending=False, axis=0)

In [0]:
sorted_df

,shortage_score,overstock_score,unbalance_score
District,,,
서초구,0.549478,0.329171,0.505417
용산구,0.451931,0.419781,0.445501
강남구,0.445992,0.43354,0.443501
종로구,0.449191,0.407013,0.440755
금천구,0.387311,0.481985,0.406246
강서구,0.384248,0.492706,0.40594
중구,0.370905,0.498222,0.396369
영등포구,0.365944,0.504231,0.393602
구로구,0.348112,0.519695,0.382429


In [0]:
path = './Data/scoring_distribution_unbalance/'
sorted_df.to_csv(path+'scores_'+str(len_period)+'days.csv', index=True, encoding='utf-8-sig')

## Based on unbalance score, select 서초구

In [0]:
data_path = './Data/district_0314-0512/'
df = pd.read_csv(data_path+'서초구'+'_0314-0512.csv', header=0, parse_dates=['collect_at', 'stock_at'], engine='python', encoding='utf-8-sig')
df

,code,name,addr,latitude,longitude,collect_at,remain_stat,stock_at
0,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-03-14 00:00:00,empty,2020-03-13 08:11:00
1,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-03-14 00:05:00,empty,2020-03-13 08:11:00
2,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-03-14 00:10:00,empty,2020-03-13 08:11:00
3,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-03-14 00:15:00,empty,2020-03-13 08:11:00
4,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-03-14 00:20:00,empty,2020-03-13 08:11:00
...,...,...,...,...,...,...,...,...
3349208,12870170,숲약국,"서울특별시 서초구 잠원로 148 107호 (잠원동, 잠원연합상가)",37.516266,127.010074,NaT,NaN,NaT
3349209,12870510,방배세명약국,서울특별시 서초구 동작대로 114 1층 (방배동),37.486800,126.982704,NaT,NaN,NaT
3349210,12870510,방배세명약국,서울특별시 서초구 동작대로 114 1층 (방배동),37.486800,126.982704,NaT,NaN,NaT
3349211,12870510,방배세명약국,서울특별시 서초구 동작대로 114 1층 (방배동),37.486800,126.982704,NaT,NaN,NaT


In [0]:
df

,code,name,addr,latitude,longitude,collect_at,remain_stat,stock_at,date
0,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-03-14 00:00:00,empty,2020-03-13 08:11:00,2020-03-13
1,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-03-14 00:05:00,empty,2020-03-13 08:11:00,2020-03-13
2,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-03-14 00:10:00,empty,2020-03-13 08:11:00,2020-03-13
3,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-03-14 00:15:00,empty,2020-03-13 08:11:00,2020-03-13
4,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-03-14 00:20:00,empty,2020-03-13 08:11:00,2020-03-13
...,...,...,...,...,...,...,...,...,...
3349208,12870170,숲약국,"서울특별시 서초구 잠원로 148 107호 (잠원동, 잠원연합상가)",37.516266,127.010074,NaT,NaN,NaT,NaT
3349209,12870510,방배세명약국,서울특별시 서초구 동작대로 114 1층 (방배동),37.486800,126.982704,NaT,NaN,NaT,NaT
3349210,12870510,방배세명약국,서울특별시 서초구 동작대로 114 1층 (방배동),37.486800,126.982704,NaT,NaN,NaT,NaT
3349211,12870510,방배세명약국,서울특별시 서초구 동작대로 114 1층 (방배동),37.486800,126.982704,NaT,NaN,NaT,NaT


In [0]:
import datetime as dt
df_0511 = df[df['date']==dt.date(2020, 5, 11)]
df_0511

,code,name,addr,latitude,longitude,collect_at,remain_stat,stock_at,date
15939,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-05-11 08:15:00,plenty,2020-05-11 08:11:00,2020-05-11
15940,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-05-11 08:20:00,plenty,2020-05-11 08:11:00,2020-05-11
15941,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-05-11 08:25:00,plenty,2020-05-11 08:11:00,2020-05-11
15942,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-05-11 08:30:00,plenty,2020-05-11 08:11:00,2020-05-11
15943,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-05-11 08:35:00,plenty,2020-05-11 08:11:00,2020-05-11
...,...,...,...,...,...,...,...,...,...
3349196,12869414,8층약국,서울특별시 서초구 서초중앙로 125 로이어즈타워 8층 803호 (서초동),37.492993,127.013451,2020-05-11 23:35:00,empty,2020-05-11 08:54:00,2020-05-11
3349197,12869414,8층약국,서울특별시 서초구 서초중앙로 125 로이어즈타워 8층 803호 (서초동),37.492993,127.013451,2020-05-11 23:40:00,empty,2020-05-11 08:54:00,2020-05-11
3349198,12869414,8층약국,서울특별시 서초구 서초중앙로 125 로이어즈타워 8층 803호 (서초동),37.492993,127.013451,2020-05-11 23:45:00,empty,2020-05-11 08:54:00,2020-05-11
3349199,12869414,8층약국,서울특별시 서초구 서초중앙로 125 로이어즈타워 8층 803호 (서초동),37.492993,127.013451,2020-05-11 23:50:00,empty,2020-05-11 08:54:00,2020-05-11


In [0]:
df_05 = df_0511[df_0511['collect_at'] >= '2020-05-11 14:00:00']
df_05 = df_05[df_05['collect_at'] <= '2020-05-11 15:00:00']
df_05

,code,name,addr,latitude,longitude,collect_at,remain_stat,stock_at,date
16008,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-05-11 14:00:00,plenty,2020-05-11 08:11:00,2020-05-11
16009,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-05-11 14:05:00,plenty,2020-05-11 08:11:00,2020-05-11
16010,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-05-11 14:10:00,plenty,2020-05-11 08:11:00,2020-05-11
16011,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-05-11 14:15:00,plenty,2020-05-11 08:11:00,2020-05-11
16012,11800208,서초삼성약국,서울특별시 서초구 서초대로74길 23 102호 (서초동),37.495623,127.027107,2020-05-11 14:20:00,plenty,2020-05-11 08:11:00,2020-05-11
...,...,...,...,...,...,...,...,...,...
3349089,12869414,8층약국,서울특별시 서초구 서초중앙로 125 로이어즈타워 8층 803호 (서초동),37.492993,127.013451,2020-05-11 14:40:00,plenty,2020-05-11 08:54:00,2020-05-11
3349090,12869414,8층약국,서울특별시 서초구 서초중앙로 125 로이어즈타워 8층 803호 (서초동),37.492993,127.013451,2020-05-11 14:45:00,plenty,2020-05-11 08:54:00,2020-05-11
3349091,12869414,8층약국,서울특별시 서초구 서초중앙로 125 로이어즈타워 8층 803호 (서초동),37.492993,127.013451,2020-05-11 14:50:00,plenty,2020-05-11 08:54:00,2020-05-11
3349092,12869414,8층약국,서울특별시 서초구 서초중앙로 125 로이어즈타워 8층 803호 (서초동),37.492993,127.013451,2020-05-11 14:55:00,plenty,2020-05-11 08:54:00,2020-05-11


In [0]:
plenty = df_05[df_05['remain_stat']=='plenty']
plenty['code'].unique()
len(plenty['code'].unique())

131

In [0]:
some = df_05[df_05['remain_stat']=='some']
len(some['code'].unique())

68

In [0]:
few = df_05[df_05['remain_stat']=='few']
len(few['code'].unique())

31

In [0]:
empty = df_05[df_05['remain_stat']=='empty']
len(empty['code'].unique())

13